This was an attempt to use the Madgwick algorithm to use an acceleratometer to estimate velocity and position.  It didn't work: the data from the accelerometer just is not accurate enough to support the integration necessary, and so the results are mostly nonsense...

In [ ]:
!pip install numpy --upgrade
!pip install imufusion

In [ ]:
import matplotlib.pyplot as pyplot
import numpy
import sys
import imufusion

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

In [ ]:
import pandas as pd
home = '/gdrive/MyDrive/Engineering/Code/Across_class_notebooks/'
#filename = 'no_mvmt_clean'
filename = 'acc_data_9'
#filename = 'example_yaw1'
file = home + filename + ".csv"
data = numpy.genfromtxt(file,delimiter = ',', skip_header = 1)

In [ ]:
# Import sensor data
timestamp = data[:, 0]
accelerometer = data[:, 1:4]
gyroscope = data[:, 4:7]


In [ ]:
# Process sensor data
ahrs = imufusion.Ahrs()
euler = numpy.empty((len(timestamp), 3))

# Create an array of Euler (position) angles with respect to
for index in range(len(timestamp)):
    # dt is the third argument (1/sampling rate)
    ahrs.update_no_magnetometer(gyroscope[index], accelerometer[index], 1 / 25)
    euler[index] = ahrs.quaternion.to_euler()

In [ ]:
# Plot sensor data
_, axes = pyplot.subplots(nrows=3, sharex=True)

axes[0].plot(timestamp, gyroscope[:, 0], "tab:red", label="X")
axes[0].plot(timestamp, gyroscope[:, 1], "tab:green", label="Y")
axes[0].plot(timestamp, gyroscope[:, 2], "tab:blue", label="Z")
axes[0].set_title("Gyroscope")
axes[0].set_ylabel("Degrees/s")
axes[0].grid()
#axes[0].legend()

axes[1].plot(timestamp, accelerometer[:, 0], "tab:red", label="X")
axes[1].plot(timestamp, accelerometer[:, 1], "tab:green", label="Y")
axes[1].plot(timestamp, accelerometer[:, 2], "tab:blue", label="Z")
axes[1].set_title("Accelerometer")
axes[1].set_ylabel("g")
axes[1].grid()
axes[1].legend()

# Plot Euler angles
# 'Roll' is the angular motion around the x-axis
#       where x-axis is parallel to short side of MPU (CCW +)
# 'Pitch' is the motion around the y-axis (parallel to long, CCW +)
# 'Yaw' is around the z-axis (CCW +)
axes[2].plot(timestamp, euler[:, 0], "tab:red", label="Roll")
axes[2].plot(timestamp, euler[:, 1], "tab:green", label="Pitch")
axes[2].plot(timestamp, euler[:, 2], "tab:blue", label="Yaw")
axes[2].set_title("Euler angles")
axes[2].set_xlabel("Seconds")
axes[2].set_ylabel("Degrees")
axes[2].grid()
#axes[2].legend()